In [1]:
import sys
sys.path.append('../..') 
from typing import Any
import cupy as cp
import torch

from Deep_gpu import MyModel
from Deep_gpu import ReLU
from Deep_gpu import MeanSquaredError, CrossEntropyLoss
from Deep_gpu import GradientDescentOptimizer
from Deep_gpu import Regularization

import torchvision
import torchvision.transforms as transforms
from torchvision import datasets, transforms

import matplotlib.pyplot as plt

In [2]:
"""
多层感知机进行MINIST手写数据集的识别
"""
photo_nums = 64

transform = transforms.Compose([transforms.ToTensor()])

# 获取MNIST数据集
# windows
# train_dataset = datasets.MNIST(root='G:\学习文件\python学习\CODE\data', train=True, transform=transform, download=True)
# test_dataset = datasets.MNIST(root='G:\学习文件\python学习\CODE\data', train=False, transform=transform, download=True)
# linux
train_dataset = datasets.MNIST(root='/media/xc/学习/学习文件/python学习/CODE/data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='/media/xc/学习/学习文件/python学习/CODE/data', train=False, transform=transform, download=True)

# 定义数据加载器
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=photo_nums, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=photo_nums, shuffle=False)


In [3]:
loss_fn = CrossEntropyLoss()
reg = Regularization(0.001)

layer_dict = {
    'first': {
        'type': 'linear',
        'in_features': 784,
        'out_features': 128,
        'activation': "ReLU"
    },
    'output': {
        'type': 'linear',
        'in_features': 128,
        'out_features': 10,
        'activation': "Softmax",
        'loss_fn': loss_fn
    }
}

op = GradientDescentOptimizer(lr=0.001, max_iterations=1)
model = MyModel(layers_dict=layer_dict, optimizer = op, regularization=reg)

In [4]:
epochs = 20
for epoch in range(epochs):
    k = 0
    for batch in train_loader:
        inputs, labels = batch

        x = inputs.numpy()
        y = labels.numpy()
        x = x.reshape(photo_nums, -1)
        x = cp.asarray(x)
        y_one_hot = cp.zeros((len(y), 10))
        for i in range(len(y)):
            y_one_hot[i, y[i]] = 1

        if x.shape[1] == 784:
            model.fit(x, y_one_hot)

        k += 1
        if k >= 500:
            break
    running_loss = model.loss[-1]
    print(f"{64*500} photos, Epoch {epoch + 1}, Loss: {running_loss}")

32000 photos, Epoch 1, Loss: 9.352265797073656
32000 photos, Epoch 2, Loss: 6.508026673538154
32000 photos, Epoch 3, Loss: 6.588470234581495
32000 photos, Epoch 4, Loss: 5.802239805325227
32000 photos, Epoch 5, Loss: 4.801913083784061
32000 photos, Epoch 6, Loss: 4.156478678756235
32000 photos, Epoch 7, Loss: 5.06568730979461
32000 photos, Epoch 8, Loss: 4.14465330464245
32000 photos, Epoch 9, Loss: 2.3025850929940455
32000 photos, Epoch 10, Loss: 2.7631021115928545
32000 photos, Epoch 11, Loss: 1.3815510557964275
32000 photos, Epoch 12, Loss: 4.605170185988091
32000 photos, Epoch 13, Loss: 1.8420680743952365
32000 photos, Epoch 14, Loss: 3.684136148790473
32000 photos, Epoch 15, Loss: 0.9210340371976182
32000 photos, Epoch 16, Loss: 2.7631021115928545
32000 photos, Epoch 17, Loss: 0.0
32000 photos, Epoch 18, Loss: 0.4605170185988091
32000 photos, Epoch 19, Loss: 1.3815510557964275
32000 photos, Epoch 20, Loss: 2.3025850929940455


In [6]:
acc = 0.0
current = 0.0
loss = 0.0
n = 0
for batch in test_loader:
    inputs, labels = batch

    x = inputs.numpy()
    y = labels.numpy()
    x = cp.asarray(x)
    y = cp.asarray(y)
    x = x.reshape(photo_nums, -1)

    if x.shape[1] == 784:
        predict = model.predict(x)
        loss += model.loss[-1]
        n += 1
        max_indices = predict.argmax(axis=0)  # 沿着列的方向找到最大值的索引, 代表模型预测的分类
        acc = cp.sum(max_indices == y) / predict.shape[1]
        current += acc

print(f"test_acc: {current/n}, test_loss: {loss/n}")

test_acc: 0.9185697115384616, test_loss: 2.30258509299404
